
---

# **Chapter 14: BLoC Pattern (Business Logic Component)**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the BLoC architectural pattern and its separation of concerns
- Define Events as user intentions and States as immutable representations
- Implement BLoCs using the flutter_bloc library
- Use BlocBuilder, BlocListener, and BlocConsumer widgets effectively
- Integrate the Repository pattern with BLoC architecture
- Persist BLoC state using HydratedBloc
- Implement Cubit for simpler state management scenarios
- Test BLoCs using the bloc_test library
- Apply BLoC best practices for scalable applications

---

## **Prerequisites**

- Completed Chapter 6: Asynchronous Programming
- Completed Chapter 13: Riverpod (understanding of state management concepts)
- Understanding of Streams and Stream transformations
- Knowledge of immutable data patterns
- Familiarity with architectural patterns (MVC, MVVM)

---

## **14.1 BLoC Pattern Overview**

The BLoC (Business Logic Component) pattern, introduced by Google at DartConf 2018, separates business logic from the UI layer. It uses Streams to handle data flow, making it predictable, testable, and scalable.

### **BLoC Architecture**

```dart
// The BLoC pattern follows a unidirectional data flow:

// ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
// │     UI      │────▶│    BLoC     │────▶│ Repository  │
// │  (Widgets)  │     │  (Logic)    │     │  (Data)     │
// └─────────────┘     └──────┬──────┘     └─────────────┘
//                            │
//                            ▼
//                     ┌─────────────┐
//                     │    State    │
//                     │  (Output)   │
//                     └──────┬──────┘
//                            │
//                            ▼
//                     ┌─────────────┐
//                     │     UI      │
//                     │  (Rebuild)  │
//                     └─────────────┘

// Key principles:
// 1. Events flow INTO the BLoC (user actions)
// 2. States flow OUT of the BLoC (UI updates)
// 3. BLoC contains all business logic
// 4. UI is passive - it displays state and dispatches events
```

**Explanation:**

- **Unidirectional data flow**: Events go in one direction (UI → BLoC), States come out the other direction (BLoC → UI). This makes data flow predictable and debugging easier.
- **Separation of concerns**: UI knows nothing about business logic; BLoC knows nothing about how data is displayed; Repository knows nothing about business rules.
- **Testability**: Because BLoCs are pure Dart classes (no Flutter dependencies in core logic), they can be unit tested without widget trees.
- **Reusability**: The same BLoC can be used across different UIs (mobile, web, desktop) because it's decoupled from the presentation layer.
- **Predictability**: Given the same sequence of events, a BLoC always produces the same states, making the app deterministic.

### **Core Components**

```dart
import 'package:flutter_bloc/flutter_bloc.dart';

// 1. EVENTS - Represent user intentions/actions
// Events are dispatched by the UI and processed by the BLoC
abstract class CounterEvent {}

class CounterIncrementPressed extends CounterEvent {
  final int amount;
  
  CounterIncrementPressed({this.amount = 1});
  
  @override
  String toString() => 'CounterIncrementPressed(amount: $amount)';
}

class CounterDecrementPressed extends CounterEvent {
  final int amount;
  
  CounterDecrementPressed({this.amount = 1});
}

class CounterResetPressed extends CounterEvent {}

// 2. STATES - Immutable representations of the UI at a point in time
// States are emitted by the BLoC and consumed by the UI
class CounterState {
  final int count;
  final bool isLoading;
  final String? errorMessage;
  
  const CounterState({
    this.count = 0,
    this.isLoading = false,
    this.errorMessage,
  });
  
  // Factory constructors for common states
  factory CounterState.initial() => const CounterState();
  
  factory CounterState.loading(int currentCount) => 
      CounterState(count: currentCount, isLoading: true);
  
  factory CounterState.success(int count) => 
      CounterState(count: count);
  
  factory CounterState.failure(int currentCount, String error) => 
      CounterState(count: currentCount, errorMessage: error);
  
  // Copy method for immutable updates
  CounterState copyWith({
    int? count,
    bool? isLoading,
    String? errorMessage,
  }) {
    return CounterState(
      count: count ?? this.count,
      isLoading: isLoading ?? this.isLoading,
      errorMessage: errorMessage, // Can be set to null explicitly
    );
  }
  
  @override
  String toString() => 
      'CounterState(count: $count, isLoading: $isLoading, error: $errorMessage)';
}

// 3. BLoC - Business Logic Component
// Extends Bloc<Event, State>
class CounterBloc extends Bloc<CounterEvent, CounterState> {
  // Constructor calls super with initial state
  CounterBloc() : super(CounterState.initial()) {
    // Register event handlers
    // When CounterIncrementPressed is received, call _onIncrement
    on<CounterIncrementPressed>(_onIncrement);
    on<CounterDecrementPressed>(_onDecrement);
    on<CounterResetPressed>(_onReset);
  }
  
  // Event handler for increment
  Future<void> _onIncrement(
    CounterIncrementPressed event,
    Emitter<CounterState> emit,
  ) async {
    // Emit loading state
    emit(state.copyWith(isLoading: true));
    
    // Simulate async operation (API call, database, etc.)
    await Future.delayed(Duration(milliseconds: 500));
    
    // Calculate new state
    final newCount = state.count + event.amount;
    
    // Emit success state
    emit(CounterState.success(newCount));
  }
  
  void _onDecrement(
    CounterDecrementPressed event,
    Emitter<CounterState> emit,
  ) {
    // Synchronous operation
    final newCount = state.count - event.amount;
    emit(state.copyWith(count: newCount));
  }
  
  void _onReset(CounterResetPressed event, Emitter<CounterState> emit) {
    emit(CounterState.initial());
  }
  
  @override
  void onChange(Change<CounterState> change) {
    // Called whenever state changes - good for logging
    print('State changed: ${change.currentState} -> ${change.nextState}');
    super.onChange(change);
  }
  
  @override
  void onTransition(Transition<CounterEvent, CounterState> transition) {
    // Called on every event processing - includes event and state change
    print('Transition: ${transition.event} -> ${transition.nextState}');
    super.onTransition(transition);
  }
  
  @override
  void onError(Object error, StackTrace stackTrace) {
    // Called when an error occurs in the BLoC
    print('Error: $error');
    super.onError(error, stackTrace);
  }
}
```

**Explanation:**

- **Events**: Plain classes (often abstract base with concrete implementations) that represent something that happened or user intention. Events flow INTO the BLoC.
- **States**: Immutable classes that represent the UI at a specific moment. They contain all data needed to render the UI. States flow OUT of the BLoC.
- **`extends Bloc<Event, State>`**: The BLoC class is generic over Event and State types. This provides type safety.
- **`super(initialState)`**: The constructor must call super with the initial state that the UI should display before any events are processed.
- **`on<EventType>()`**: Registers a handler function for a specific event type. When that event is added to the BLoC, the handler is called.
- **`Emitter<State>`**: The `emit` function passed to handlers. It's used to output new states. You cannot emit after an `await` without the `Emitter` parameter.
- **`emit()`**: Updates the BLoC's state and notifies all listeners. States must be emitted in response to events.
- **Lifecycle hooks**: `onChange`, `onTransition`, and `onError` allow you to observe BLoC behavior for debugging, logging, or analytics.

---

## **14.2 Flutter Integration**

The flutter_bloc library provides widgets that connect BLoCs to the Flutter widget tree.

### **Basic BLoC Widget Integration**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_bloc/flutter_bloc.dart';

void main() {
  runApp(MyApp());
}

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'BLoC Demo',
      home: BlocProvider(
        // Create the BLoC instance
        create: (context) => CounterBloc(),
        // BlocProvider makes the BLoC available to child widgets
        child: CounterScreen(),
      ),
    );
  }
}

class CounterScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('BLoC Counter')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            // BlocBuilder rebuilds when state changes
            BlocBuilder<CounterBloc, CounterState>(
              builder: (context, state) {
                // builder is called with current state
                // Rebuilds automatically when state changes
                return Text(
                  '${state.count}',
                  style: Theme.of(context).textTheme.headline2,
                );
              },
            ),
            
            SizedBox(height: 20),
            
            // Show loading indicator based on state
            BlocBuilder<CounterBloc, CounterState>(
              buildWhen: (previous, current) {
                // Optional optimization: only rebuild when isLoading changes
                return previous.isLoading != current.isLoading;
              },
              builder: (context, state) {
                if (state.isLoading) {
                  return CircularProgressIndicator();
                }
                return SizedBox.shrink(); // Empty space when not loading
              },
            ),
            
            // Show error message
            BlocBuilder<CounterBloc, CounterState>(
              builder: (context, state) {
                if (state.errorMessage != null) {
                  return Text(
                    state.errorMessage!,
                    style: TextStyle(color: Colors.red),
                  );
                }
                return SizedBox.shrink();
              },
            ),
          ],
        ),
      ),
      floatingActionButton: Column(
        mainAxisAlignment: MainAxisAlignment.end,
        children: [
          FloatingActionButton(
            onPressed: () {
              // Access the BLoC and add an event
              // context.read<T>() gets the BLoC without listening
              context.read<CounterBloc>().add(CounterIncrementPressed());
            },
            child: Icon(Icons.add),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            onPressed: () {
              context.read<CounterBloc>().add(CounterDecrementPressed());
            },
            child: Icon(Icons.remove),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            onPressed: () {
              context.read<CounterBloc>().add(CounterResetPressed());
            },
            child: Icon(Icons.refresh),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`BlocProvider`**: Creates and provides a BLoC to its subtree. Similar to Provider's ChangeNotifierProvider, but specifically for BLoCs. It automatically closes the BLoC when the widget is disposed.
- **`create` callback**: Called when the BLoC is first needed. Returns the BLoC instance.
- **`BlocBuilder<BLoC, State>`**: Widget that rebuilds when the BLoC emits new states. Takes a `builder` function that receives the current state.
- **`buildWhen`**: Optional callback that receives previous and current state. Return `true` to rebuild, `false` to skip. Useful for optimization.
- **`context.read<CounterBloc>()`**: Gets the BLoC instance without subscribing to changes. Use this to dispatch events in callbacks.
- **`bloc.add(event)`**: Sends an event to the BLoC for processing. The BLoC will handle it according to registered event handlers.
- **Multiple BlocBuilders**: You can have multiple BlocBuilders listening to the same BLoC. Each can listen to different parts of the state using `buildWhen`.

### **BlocListener for Side Effects**

```dart
class LoginScreen extends StatelessWidget {
  final _emailController = TextEditingController();
  final _passwordController = TextEditingController();
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: BlocListener<AuthBloc, AuthState>(
        // BlocListener listens to state changes but doesn't rebuild
        // Use it for side effects: navigation, snackbars, dialogs
        listener: (context, state) {
          // Called whenever state changes
          if (state is AuthSuccess) {
            // Navigate to home screen on success
            Navigator.of(context).pushReplacement(
              MaterialPageRoute(builder: (_) => HomeScreen()),
            );
          } else if (state is AuthFailure) {
            // Show error snackbar
            ScaffoldMessenger.of(context).showSnackBar(
              SnackBar(
                content: Text(state.error),
                backgroundColor: Colors.red,
              ),
            );
          }
        },
        child: Padding(
          padding: EdgeInsets.all(16),
          child: Column(
            mainAxisAlignment: MainAxisAlignment.center,
            children: [
              TextField(
                controller: _emailController,
                decoration: InputDecoration(labelText: 'Email'),
              ),
              TextField(
                controller: _passwordController,
                decoration: InputDecoration(labelText: 'Password'),
                obscureText: true,
              ),
              SizedBox(height: 20),
              BlocBuilder<AuthBloc, AuthState>(
                builder: (context, state) {
                  if (state is AuthLoading) {
                    return CircularProgressIndicator();
                  }
                  return ElevatedButton(
                    onPressed: () {
                      // Dispatch login event
                      context.read<AuthBloc>().add(
                        AuthLoginRequested(
                          email: _emailController.text,
                          password: _passwordController.text,
                        ),
                      );
                    },
                    child: Text('Login'),
                  );
                },
              ),
            ],
          ),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`BlocListener`**: Listens to state changes and calls the `listener` callback, but doesn't rebuild the child widget. Perfect for one-time actions like navigation or showing snackbars.
- **Side effects**: Navigation, showing dialogs, snackbars, or triggering analytics should happen in BlocListener, not BlocBuilder.
- **Single execution**: The listener runs once per state change, even if the widget rebuilds multiple times.
- **Child widget**: The child is not rebuilt when state changes (unlike BlocBuilder).

### **BlocConsumer (Builder + Listener)**

```dart
class ProductScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return BlocConsumer<ProductBloc, ProductState>(
      // BlocConsumer combines BlocBuilder and BlocListener
      // It has both builder and listener properties
      
      listenWhen: (previous, current) {
        // Optional: only call listener when this returns true
        // Useful to avoid duplicate snackbars
        return previous.error != current.error && current.error != null;
      },
      listener: (context, state) {
        // Handle side effects
        if (state.error != null) {
          ScaffoldMessenger.of(context).showSnackBar(
            SnackBar(content: Text(state.error!)),
          );
        }
      },
      buildWhen: (previous, current) {
        // Optional: only rebuild when products change
        return previous.products != current.products;
      },
      builder: (context, state) {
        // Build UI based on state
        if (state.isLoading) {
          return Center(child: CircularProgressIndicator());
        }
        
        if (state.products.isEmpty) {
          return Center(child: Text('No products'));
        }
        
        return ListView.builder(
          itemCount: state.products.length,
          itemBuilder: (context, index) {
            return ProductTile(product: state.products[index]);
          },
        );
      },
    );
  }
}
```

**Explanation:**

- **`BlocConsumer`**: Combines `BlocListener` and `BlocBuilder` into one widget. Use when you need both UI updates and side effects.
- **`listenWhen`**: Controls when the listener callback is called. Prevents duplicate snackbars when rebuilding.
- **`buildWhen`**: Controls when the builder is called. Optimizes performance by skipping unnecessary rebuilds.
- **When to use**: Use BlocConsumer when a widget needs to both rebuild based on state AND perform side effects (like showing errors). If you only need one or the other, use the specific widget for clarity.

---

## **14.3 Repository Pattern Integration**

The Repository pattern abstracts data access, allowing BLoCs to remain agnostic of data sources (API, database, cache).

### **Repository Implementation**

```dart
// Domain layer - Entity
class User {
  final String id;
  final String email;
  final String name;
  
  const User({
    required this.id,
    required this.email,
    required this.name,
  });
  
  factory User.fromJson(Map<String, dynamic> json) {
    return User(
      id: json['id'],
      email: json['email'],
      name: json['name'],
    );
  }
}

// Data layer - Repository interface
abstract class UserRepository {
  Future<User> getUser(String id);
  Future<List<User>> getUsers();
  Future<User> createUser(String email, String name);
  Future<void> updateUser(String id, {String? name, String? email});
  Future<void> deleteUser(String id);
}

// Data layer - Implementation
class ApiUserRepository implements UserRepository {
  final Dio dio; // HTTP client
  
  ApiUserRepository({required this.dio});
  
  @override
  Future<User> getUser(String id) async {
    try {
      final response = await dio.get('/users/$id');
      return User.fromJson(response.data);
    } on DioError catch (e) {
      throw RepositoryException('Failed to fetch user: ${e.message}');
    }
  }
  
  @override
  Future<List<User>> getUsers() async {
    final response = await dio.get('/users');
    return (response.data as List)
        .map((json) => User.fromJson(json))
        .toList();
  }
  
  @override
  Future<User> createUser(String email, String name) async {
    final response = await dio.post('/users', data: {
      'email': email,
      'name': name,
    });
    return User.fromJson(response.data);
  }
  
  @override
  Future<void> updateUser(String id, {String? name, String? email}) async {
    await dio.patch('/users/$id', data: {
      if (name != null) 'name': name,
      if (email != null) 'email': email,
    });
  }
  
  @override
  Future<void> deleteUser(String id) async {
    await dio.delete('/users/$id');
  }
}

// Exception class
class RepositoryException implements Exception {
  final String message;
  RepositoryException(this.message);
  
  @override
  String toString() => 'RepositoryException: $message';
}
```

**Explanation:**

- **Repository Pattern**: Abstracts data operations. The BLoC doesn't know if data comes from API, local database, or cache.
- **Interface/Implementation**: Define an abstract repository class, then implement it for different data sources (ApiUserRepository, LocalUserRepository, MockUserRepository).
- **Error handling**: Repository catches low-level errors (DioError) and converts them to domain-specific exceptions.
- **Dependency injection**: Repository is injected into BLoC via constructor, allowing easy swapping for testing.

### **BLoC with Repository**

```dart
// Events
abstract class UserEvent {}

class UserLoadRequested extends UserEvent {
  final String userId;
  UserLoadRequested(this.userId);
}

class UserListRequested extends UserEvent {}

class UserCreateRequested extends UserEvent {
  final String email;
  final String name;
  UserCreateRequested(this.email, this.name);
}

class UserUpdateRequested extends UserEvent {
  final String userId;
  final String? name;
  final String? email;
  UserUpdateRequested(this.userId, {this.name, this.email});
}

class UserDeleteRequested extends UserEvent {
  final String userId;
  UserDeleteRequested(this.userId);
}

// States
abstract class UserState {}

class UserInitial extends UserState {}

class UserLoadInProgress extends UserState {}

class UserLoadSuccess extends UserState {
  final User user;
  UserLoadSuccess(this.user);
}

class UserListLoadSuccess extends UserState {
  final List<User> users;
  UserListLoadSuccess(this.users);
}

class UserOperationFailure extends UserState {
  final String error;
  UserOperationFailure(this.error);
}

// BLoC
class UserBloc extends Bloc<UserEvent, UserState> {
  final UserRepository userRepository;
  
  // Repository is injected via constructor
  UserBloc({required this.userRepository}) : super(UserInitial()) {
    on<UserLoadRequested>(_onLoadUser);
    on<UserListRequested>(_onLoadUsers);
    on<UserCreateRequested>(_onCreateUser);
    on<UserUpdateRequested>(_onUpdateUser);
    on<UserDeleteRequested>(_onDeleteUser);
  }
  
  Future<void> _onLoadUser(
    UserLoadRequested event,
    Emitter<UserState> emit,
  ) async {
    emit(UserLoadInProgress());
    try {
      final user = await userRepository.getUser(event.userId);
      emit(UserLoadSuccess(user));
    } catch (e) {
      emit(UserOperationFailure(e.toString()));
    }
  }
  
  Future<void> _onLoadUsers(
    UserListRequested event,
    Emitter<UserState> emit,
  ) async {
    emit(UserLoadInProgress());
    try {
      final users = await userRepository.getUsers();
      emit(UserListLoadSuccess(users));
    } catch (e) {
      emit(UserOperationFailure(e.toString()));
    }
  }
  
  Future<void> _onCreateUser(
    UserCreateRequested event,
    Emitter<UserState> emit,
  ) async {
    emit(UserLoadInProgress());
    try {
      await userRepository.createUser(event.email, event.name);
      // Reload list after creation
      final users = await userRepository.getUsers();
      emit(UserListLoadSuccess(users));
    } catch (e) {
      emit(UserOperationFailure(e.toString()));
    }
  }
  
  Future<void> _onUpdateUser(
    UserUpdateRequested event,
    Emitter<UserState> emit,
  ) async {
    try {
      await userRepository.updateUser(
        event.userId,
        name: event.name,
        email: event.email,
      );
      // Reload user after update
      final user = await userRepository.getUser(event.userId);
      emit(UserLoadSuccess(user));
    } catch (e) {
      emit(UserOperationFailure(e.toString()));
    }
  }
  
  Future<void> _onDeleteUser(
    UserDeleteRequested event,
    Emitter<UserState> emit,
  ) async {
    try {
      await userRepository.deleteUser(event.userId);
      emit(UserInitial()); // Reset to initial state
    } catch (e) {
      emit(UserOperationFailure(e.toString()));
    }
  }
}
```

**Explanation:**

- **Dependency injection**: The BLoC receives `UserRepository` in its constructor. This allows testing with mock repositories.
- **State per operation**: Different states for loading, success, and failure. Each operation (load, create, update) has appropriate state transitions.
- **Error handling**: Try-catch blocks convert repository exceptions into failure states that the UI can display.
- **Sequential operations**: After creating a user, the BLoC automatically reloads the list to keep state consistent.

### **Wiring it all together**

```dart
void main() {
  // Initialize dependencies
  final dio = Dio(BaseOptions(baseUrl: 'https://api.example.com'));
  final userRepository = ApiUserRepository(dio: dio);
  
  runApp(MyApp(userRepository: userRepository));
}

class MyApp extends StatelessWidget {
  final UserRepository userRepository;
  
  const MyApp({Key? key, required this.userRepository}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: RepositoryProvider(
        // Provide repository to the subtree
        create: (context) => userRepository,
        child: BlocProvider(
          // Create BLoC with injected repository
          create: (context) => UserBloc(
            userRepository: context.read<UserRepository>(),
          ),
          child: UserScreen(),
        ),
      ),
    );
  }
}

class UserScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Users'),
        actions: [
          IconButton(
            icon: Icon(Icons.refresh),
            onPressed: () {
              context.read<UserBloc>().add(UserListRequested());
            },
          ),
        ],
      ),
      body: BlocConsumer<UserBloc, UserState>(
        listener: (context, state) {
          if (state is UserOperationFailure) {
            ScaffoldMessenger.of(context).showSnackBar(
              SnackBar(content: Text(state.error)),
            );
          }
        },
        builder: (context, state) {
          if (state is UserLoadInProgress) {
            return Center(child: CircularProgressIndicator());
          } else if (state is UserListLoadSuccess) {
            return ListView.builder(
              itemCount: state.users.length,
              itemBuilder: (context, index) {
                final user = state.users[index];
                return ListTile(
                  title: Text(user.name),
                  subtitle: Text(user.email),
                  trailing: IconButton(
                    icon: Icon(Icons.delete),
                    onPressed: () {
                      context.read<UserBloc>().add(
                        UserDeleteRequested(user.id),
                      );
                    },
                  ),
                );
              },
            );
          } else if (state is UserLoadSuccess) {
            // Show single user detail
            return UserDetailView(user: state.user);
          }
          return Center(
            child: ElevatedButton(
              onPressed: () {
                context.read<UserBloc>().add(UserListRequested());
              },
              child: Text('Load Users'),
            ),
          );
        },
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          // Show dialog to create user
          showDialog(
            context: context,
            builder: (context) => CreateUserDialog(),
          );
        },
        child: Icon(Icons.add),
      ),
    );
  }
}
```

**Explanation:**

- **`RepositoryProvider`**: Special provider from flutter_bloc that provides repositories. It's similar to Provider but designed for BLoC architecture.
- **Dependency chain**: RepositoryProvider → BlocProvider → Widgets. Each layer receives its dependencies from above.
- **Context.read**: Used to access repository when creating BLoC, and to access BLoC when dispatching events.
- **Separation**: UI doesn't know about Dio or HTTP; it only knows about UserBloc and UserState.

---

## **14.4 HydratedBloc for Persistence**

HydratedBloc automatically persists and restores BLoC state across app restarts using local storage.

### **Setting Up HydratedBloc**

```dart
import 'package:hydrated_bloc/hydrated_bloc.dart';
import 'package:path_provider/path_provider.dart';

void main() async {
  // Ensure Flutter bindings are initialized
  WidgetsFlutterBinding.ensureInitialized();
  
  // Initialize HydratedBloc storage
  HydratedBloc.storage = await HydratedStorage.build(
    storageDirectory: await getTemporaryDirectory(),
  );
  
  runApp(MyApp());
}

// Counter BLoC with persistence
class CounterBloc extends HydratedBloc<CounterEvent, CounterState> {
  CounterBloc() : super(CounterState.initial()) {
    on<CounterIncrementPressed>(_onIncrement);
    on<CounterDecrementPressed>(_onDecrement);
  }
  
  void _onIncrement(CounterIncrementPressed event, Emitter<CounterState> emit) {
    emit(state.copyWith(count: state.count + 1));
  }
  
  void _onDecrement(CounterDecrementPressed event, Emitter<CounterState> emit) {
    emit(state.copyWith(count: state.count - 1));
  }
  
  @override
  CounterState fromJson(Map<String, dynamic> json) {
    // Called when restoring state from storage
    // Convert JSON map to state object
    return CounterState(
      count: json['count'] as int,
    );
  }
  
  @override
  Map<String, dynamic> toJson(CounterState state) {
    // Called when saving state to storage
    // Convert state object to JSON map
    return {
      'count': state.count,
    };
  }
}
```

**Explanation:**

- **`HydratedBloc`**: Extends regular Bloc with automatic persistence. State is saved to local storage whenever it changes.
- **`fromJson`**: Deserialization method. Converts stored JSON back into state objects when app restarts.
- **`toJson`**: Serialization method. Converts current state to JSON for storage.
- **Storage directory**: Usually `getTemporaryDirectory()` or `getApplicationDocumentsDirectory()`.
- **Automatic**: No manual save/load calls needed. The BLoC handles persistence transparently.

### **Complex State Persistence**

```dart
// For complex states, you need proper JSON serialization
import 'package:json_annotation/json_annotation.dart';

part 'todo_state.g.dart'; // Generated file

@JsonSerializable()
class Todo {
  final String id;
  final String title;
  final bool isCompleted;
  final DateTime createdAt;
  
  Todo({
    required this.id,
    required this.title,
    this.isCompleted = false,
    required this.createdAt,
  });
  
  factory Todo.fromJson(Map<String, dynamic> json) => _$TodoFromJson(json);
  Map<String, dynamic> toJson() => _$TodoToJson(this);
  
  Todo copyWith({
    String? id,
    String? title,
    bool? isCompleted,
    DateTime? createdAt,
  }) {
    return Todo(
      id: id ?? this.id,
      title: title ?? this.title,
      isCompleted: isCompleted ?? this.isCompleted,
      createdAt: createdAt ?? this.createdAt,
    );
  }
}

@JsonSerializable()
class TodoState {
  final List<Todo> todos;
  final String? filter;
  
  TodoState({
    this.todos = const [],
    this.filter,
  });
  
  factory TodoState.fromJson(Map<String, dynamic> json) => 
      _$TodoStateFromJson(json);
  
  Map<String, dynamic> toJson() => _$TodoStateToJson(this);
  
  TodoState copyWith({
    List<Todo>? todos,
    String? filter,
  }) {
    return TodoState(
      todos: todos ?? this.todos,
      filter: filter ?? this.filter,
    );
  }
}

class TodoBloc extends HydratedBloc<TodoEvent, TodoState> {
  TodoBloc() : super(TodoState()) {
    on<TodoAdded>(_onAdded);
    on<TodoToggled>(_onToggled);
    on<TodoDeleted>(_onDeleted);
    on<TodoFilterChanged>(_onFilterChanged);
  }
  
  @override
  TodoState fromJson(Map<String, dynamic> json) {
    try {
      return TodoState.fromJson(json);
    } catch (_) {
      // Return initial state if JSON is corrupted
      return TodoState();
    }
  }
  
  @override
  Map<String, dynamic> toJson(TodoState state) {
    try {
      return state.toJson();
    } catch (_) {
      return {};
    }
  }
  
  void _onAdded(TodoAdded event, Emitter<TodoState> emit) {
    final newTodo = Todo(
      id: DateTime.now().toString(),
      title: event.title,
      createdAt: DateTime.now(),
    );
    emit(state.copyWith(todos: [...state.todos, newTodo]));
  }
  
  void _onToggled(TodoToggled event, Emitter<TodoState> emit) {
    final updatedTodos = state.todos.map((todo) {
      if (todo.id == event.id) {
        return todo.copyWith(isCompleted: !todo.isCompleted);
      }
      return todo;
    }).toList();
    emit(state.copyWith(todos: updatedTodos));
  }
  
  void _onDeleted(TodoDeleted event, Emitter<TodoState> emit) {
    final filteredTodos = state.todos.where((t) => t.id != event.id).toList();
    emit(state.copyWith(todos: filteredTodos));
  }
  
  void _onFilterChanged(TodoFilterChanged event, Emitter<TodoState> emit) {
    emit(state.copyWith(filter: event.filter));
  }
}
```

**Explanation:**

- **`json_annotation`**: Use code generation for JSON serialization with complex classes.
- **Error handling in fromJson**: Wrap in try-catch to handle corrupted storage data gracefully.
- **Deep serialization**: When using `json_serializable`, nested objects (like List<Todo>) are automatically handled if they also have fromJson/toJson methods.
- **Storage limits**: Be mindful of storage size. Don't persist large lists or binary data in HydratedBloc.

---

## **14.5 Cubit for Simple State Management**

Cubit is a simplified version of BLoC that doesn't use events. Instead, you call methods directly on the Cubit to emit states.

### **When to Use Cubit vs BLoC**

```dart
import 'package:flutter_bloc/flutter_bloc.dart';

// Use Cubit when:
// - Logic is simple and doesn't require event history
// - You don't need event transformations (debounce, throttle)
// - You want less boilerplate (no Event classes)

// Use BLoC when:
// - You need to track event history (undo/redo)
// - You need event transformations (debounce search input)
// - Multiple events can trigger the same state transition
// - You need event-specific logic (e.g., different handling for UserRefresh vs UserLoad)

// CUBIT EXAMPLE
class CounterCubit extends Cubit<int> {
  // Constructor takes initial state
  CounterCubit() : super(0);
  
  // Methods directly emit new states
  void increment() => emit(state + 1);
  void decrement() => emit(state - 1);
  void reset() => emit(0);
  
  // Can have async methods
  Future<void> incrementAsync() async {
    await Future.delayed(Duration(seconds: 1));
    emit(state + 1);
  }
}

// EQUIVALENT BLoC (more verbose)
abstract class CounterEvent {}
class IncrementPressed extends CounterEvent {}
class DecrementPressed extends CounterEvent {}

class CounterBloc extends Bloc<CounterEvent, int> {
  CounterBloc() : super(0) {
    on<IncrementPressed>((event, emit) => emit(state + 1));
    on<DecrementPressed>((event, emit) => emit(state - 1));
  }
}

// Using Cubit in UI (very similar to BLoC)
class CounterScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return BlocProvider(
      create: (context) => CounterCubit(),
      child: Scaffold(
        body: BlocBuilder<CounterCubit, int>(
          builder: (context, count) {
            return Center(
              child: Text('$count', style: TextStyle(fontSize: 48)),
            );
          },
        ),
        floatingActionButton: Column(
          mainAxisAlignment: MainAxisAlignment.end,
          children: [
            FloatingActionButton(
              onPressed: () => context.read<CounterCubit>().increment(),
              child: Icon(Icons.add),
            ),
            SizedBox(height: 10),
            FloatingActionButton(
              onPressed: () => context.read<CounterCubit>().decrement(),
              child: Icon(Icons.remove),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`extends Cubit<State>`**: Simpler than Bloc. No events, just methods that call `emit()`.
- **Direct method calls**: UI calls methods directly (`cubit.increment()`) instead of adding events (`bloc.add(IncrementPressed())`).
- **Less boilerplate**: No need to define Event classes for simple scenarios.
- **Same widgets**: Use the same BlocBuilder, BlocListener, BlocConsumer widgets with Cubit.
- **When to choose**: Cubit is great for simple feature toggles, counters, theme switches. BLoC is better for complex workflows like authentication flows, shopping carts, or multi-step forms.

---

## **14.6 Testing BLoCs**

The bloc_test library provides utilities for testing BLoCs with predictable state sequences.

### **Unit Testing BLoCs**

```dart
import 'package:bloc_test/bloc_test.dart';
import 'package:flutter_test/flutter_test.dart';
import 'package:mocktail/mocktail.dart';

// Mock repository for testing
class MockUserRepository extends Mock implements UserRepository {}

void main() {
  group('CounterBloc', () {
    late CounterBloc counterBloc;
    
    setUp(() {
      counterBloc = CounterBloc();
    });
    
    tearDown(() {
      counterBloc.close(); // Clean up
    });
    
    test('initial state is 0', () {
      expect(counterBloc.state, CounterState.initial());
    });
    
    blocTest<CounterBloc, CounterState>(
      'emits [loading, success] when increment is added',
      build: () => counterBloc,
      act: (bloc) => bloc.add(CounterIncrementPressed()),
      expect: () => [
        CounterState(isLoading: true, count: 0),
        CounterState(isLoading: false, count: 1),
      ],
    );
    
    blocTest<CounterBloc, CounterState>(
      'emits decremented state when decrement is added',
      build: () => counterBloc,
      seed: () => CounterState(count: 1), // Start with count=1
      act: (bloc) => bloc.add(CounterDecrementPressed()),
      expect: () => [
        CounterState(count: 0),
      ],
    );
  });
  
  group('UserBloc', () {
    late MockUserRepository userRepository;
    late UserBloc userBloc;
    
    setUp(() {
      userRepository = MockUserRepository();
      userBloc = UserBloc(userRepository: userRepository);
    });
    
    tearDown(() {
      userBloc.close();
    });
    
    blocTest<UserBloc, UserState>(
      'emits [loading, success] when user loaded successfully',
      build: () {
        // Setup mock
        when(() => userRepository.getUser('1'))
            .thenAnswer((_) async => User(id: '1', name: 'Test', email: 'test@test.com'));
        return userBloc;
      },
      act: (bloc) => bloc.add(UserLoadRequested('1')),
      expect: () => [
        UserLoadInProgress(),
        UserLoadSuccess(User(id: '1', name: 'Test', email: 'test@test.com')),
      ],
      verify: (_) {
        // Verify repository was called
        verify(() => userRepository.getUser('1')).called(1);
      },
    );
    
    blocTest<UserBloc, UserState>(
      'emits [loading, failure] when user load fails',
      build: () {
        when(() => userRepository.getUser('1'))
            .thenThrow(Exception('Network error'));
        return userBloc;
      },
      act: (bloc) => bloc.add(UserLoadRequested('1')),
      expect: () => [
        UserLoadInProgress(),
        UserOperationFailure('Exception: Network error'),
      ],
    );
  });
}
```

**Explanation:**

- **`blocTest`**: A testing utility that handles the async nature of BLoCs. It builds the BLoC, dispatches events, and verifies the sequence of states.
- **`build`**: Creates the BLoC instance. Can include mock setup.
- **`act`**: The action to perform (usually adding events).
- **`expect`**: A function that returns the expected list of states in order.
- **`seed`**: Sets an initial state before the test runs.
- **`verify`**: Additional assertions after the test runs (e.g., verifying mocks were called).
- **Mocktail**: Used to mock dependencies like repositories. `when()` sets up mock responses.
- **State equality**: States must implement `==` and `hashCode` for `expect` to work correctly (use Equatable or built_value).

---

## **Chapter Summary**

In this chapter, we covered the BLoC pattern in detail:

### **Key Takeaways:**

1. **BLoC Architecture**: Unidirectional data flow (Events → BLoC → States). Separates business logic from UI.
2. **Events**: Represent user intentions. Immutable classes that flow into the BLoC.
3. **States**: Represent UI at a point in time. Immutable classes that flow out of the BLoC.
4. **BLoC Class**: Extends `Bloc<Event, State>`. Registers event handlers with `on<EventType>()`.
5. **Flutter Integration**:
   - `BlocProvider`: Creates and provides BLoCs
   - `BlocBuilder`: Rebuilds UI when state changes
   - `BlocListener`: Handles side effects (navigation, snackbars)
   - `BlocConsumer`: Combines builder and listener
6. **Repository Pattern**: Abstract data access. Inject repositories into BLoCs via constructor for testability.
7. **HydratedBloc**: Automatic persistence of BLoC state across app restarts using `fromJson`/`toJson`.
8. **Cubit**: Simplified BLoC without events. Use for simple state management.
9. **Testing**: Use `blocTest` from bloc_test package. Mock dependencies and verify state sequences.

### **Best Practices:**

- Keep BLoCs pure (no Flutter dependencies in business logic)
- Use immutable states with `copyWith` methods
- Handle errors by emitting failure states
- Use Repository pattern for data access
- Test BLoCs with bloc_test library
- Choose Cubit for simple features, BLoC for complex workflows

---

**End of Chapter 14**

---

# **Next Chapter: Chapter 15 - Redux & MobX**

Chapter 15 will explore alternative state management solutions: Redux for predictable state container architecture with unidirectional data flow, and MobX for reactive programming with observables and automatic dependency tracking. You'll learn when to choose these patterns over BLoC or Provider.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='13. riverpod.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='15. redux_and_mobx.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
